In [31]:
from transformers import AutoConfig, AutoModelForVision2Seq
from transformers.dynamic_module_utils import get_class_from_dynamic_module
import torch

In [18]:
pretrained_model_name_or_path = "Embodied-CoT/ecot-openvla-7b-bridge"
config = AutoConfig.from_pretrained(
                pretrained_model_name_or_path,
                trust_remote_code=True
            )

/home/will/miniconda3/envs/tensorrt-ecot/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
loading configuration file config.json from cache at /home/will/.cache/huggingface/hub/models--Embodied-CoT--ecot-openvla-7b-bridge/snapshots/492b3dbf3df380f6da333f86ce06dab028176166/config.json
loading configuration file config.json from cache at /home/will/.cache/huggingface/hub/models--Embodied-CoT--ecot-openvla-7b-bridge/snapshots/492b3dbf3df380f6da333f86ce06dab028176166/config.json
Model config OpenVLAConfig {
  "_name_or_path": "Embodied-CoT/ecot-openvla-7b-bridge",
  "arch_specifier": "no-align+gelu-mlp",
  "architectures": [
    "OpenVLAForActionPrediction"
  ],
  "auto_map": {
    "AutoConfig": "Embodied-CoT/ecot-openvla-7b-bridge--configuration_prismatic.OpenVLAConfig

In [28]:
pretrained_model_name_or_path = "Embodied-CoT/ecot-openvla-7b-bridge"
config = AutoConfig.from_pretrained(
                pretrained_model_name_or_path,
                trust_remote_code=True
            )
vision_backbone_class_ref = config.auto_map[AutoModelForVision2Seq.__name__].replace("OpenVLAForActionPrediction", "PrismaticVisionBackbone")
vision_backbone_class = get_class_from_dynamic_module(
    vision_backbone_class_ref, pretrained_model_name_or_path
)

/home/will/miniconda3/envs/tensorrt-ecot/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [29]:
vision_backbone_class

transformers_modules.Embodied-CoT.ecot-openvla-7b-bridge.492b3dbf3df380f6da333f86ce06dab028176166.modeling_prismatic.PrismaticVisionBackbone

In [25]:
class_ref.replace("OpenVLAForActionPrediction", "PrismaticVisionBackbone")

'Embodied-CoT/ecot-openvla-7b-bridge--modeling_prismatic.OpenVLAForActionPrediction'

In [24]:
config

OpenVLAConfig {
  "_name_or_path": "Embodied-CoT/ecot-openvla-7b-bridge",
  "arch_specifier": "no-align+gelu-mlp",
  "architectures": [
    "OpenVLAForActionPrediction"
  ],
  "auto_map": {
    "AutoConfig": "Embodied-CoT/ecot-openvla-7b-bridge--configuration_prismatic.OpenVLAConfig",
    "AutoModelForVision2Seq": "Embodied-CoT/ecot-openvla-7b-bridge--modeling_prismatic.OpenVLAForActionPrediction"
  },
  "hf_llm_id": "lmsys/vicuna-7b-v1.5",
  "image_resize_strategy": "letterbox",
  "image_sizes": [
    224
  ],
  "llm_backbone_id": "vicuna-v15-7b",
  "llm_max_length": 2048,
  "model_type": "openvla",
  "n_action_bins": 256,
  "norm_stats": {
    "bridge_orig": {
      "action": {
        "mask": [
          true,
          true,
          true,
          true,
          true,
          true,
          false
        ],
        "max": [
          0.41691166162490845,
          0.25864794850349426,
          0.21218234300613403,
          3.122201919555664,
          1.8618112802505493,
 

In [32]:
model_name = "Embodied-CoT/ecot-openvla-7b-bridge"
vla = AutoModelForVision2Seq.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    trust_remote_code=True,
)

/home/will/miniconda3/envs/tensorrt-ecot/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
loading configuration file config.json from cache at /home/will/.cache/huggingface/hub/models--Embodied-CoT--ecot-openvla-7b-bridge/snapshots/492b3dbf3df380f6da333f86ce06dab028176166/config.json
loading configuration file config.json from cache at /home/will/.cache/huggingface/hub/models--Embodied-CoT--ecot-openvla-7b-bridge/snapshots/492b3dbf3df380f6da333f86ce06dab028176166/config.json
Model config OpenVLAConfig {
  "_name_or_path": "Embodied-CoT/ecot-openvla-7b-bridge",
  "arch_specifier": "no-align+gelu-mlp",
  "architectures": [
    "OpenVLAForActionPrediction"
  ],
  "auto_map": {
    "AutoConfig": "Embodied-CoT/ecot-openvla-7b-bridge--configuration_prismatic.OpenVLAConfig

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

All model checkpoint weights were used when initializing OpenVLAForActionPrediction.

All the weights of OpenVLAForActionPrediction were initialized from the model checkpoint at Embodied-CoT/ecot-openvla-7b-bridge.
If your task is similar to the task the model of the checkpoint was trained on, you can already use OpenVLAForActionPrediction for predictions without further training.
loading configuration file generation_config.json from cache at /home/will/.cache/huggingface/hub/models--Embodied-CoT--ecot-openvla-7b-bridge/snapshots/492b3dbf3df380f6da333f86ce06dab028176166/generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 2,
  "pad_token_id": 32000
}



In [33]:
vla.vision_backbone

PrismaticVisionBackbone(
  (featurizer): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 1152, kernel_size=(14, 14), stride=(14, 14))
      (norm): Identity()
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (patch_drop): Identity()
    (norm_pre): Identity()
    (blocks): Sequential(
      (0): Block(
        (norm1): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=1152, out_features=3456, bias=True)
          (q_norm): Identity()
          (k_norm): Identity()
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=1152, out_features=1152, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (ls1): Identity()
        (drop_path1): Identity()
        (norm2): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=1152, out_features=4304, bias=True)
          (act): G

In [34]:
vla.projector

PrismaticProjector(
  (fc1): Linear(in_features=1152, out_features=4096, bias=True)
  (fc2): Linear(in_features=4096, out_features=4096, bias=True)
  (act_fn1): GELU(approximate='none')
)